# phitigra: a simple graph editor

[SageMath](https://www.sagemath.org/) has a large set of functions for [graph theory](https://doc.sagemath.org/html/en/reference/graphs/index.html). Defining graphs by hand can however be complicated as vertices and edges are added with the command line.
This package is an editor that allows to define or change graphs using the mouse. It has the form of a [Jupyter](https://jupyter.org/) widget.

## Getting started


In [2]:
from src.phitigra import GraphEditor

The editor widget is a `GraphEditor` object. By default the canvas is empty; you can add vertices and edges by clicking on *add vertex or edge* and clicking on the canvas.

In [ ]:
editor = GraphEditor()
editor.show()

It is also possible to plot (and later edit) an already existing graph. Note that the two instances of the editor are completely independent.

In [ ]:
G = graphs.PetersenGraph()
editor2 = GraphEditor(G)
editor2.show()

Now you can move vertices, change their color, etc. The graph drawn can be accessed with `.graph`. It is the same object as the graph given when creating the widget.

In [ ]:
editor2.graph

In [ ]:
editor2.graph is G

A copy of the drawn graph can be obtained as follows:

In [ ]:
H = editor2.get_graph()
H == G and not H is G

### Application 1: testing a conjecture

In [ ]:
def conjecture(G):
    return not G.is_vertex_transitive() or G.is_hamiltonian()

Let us [conjecture](https://en.wikipedia.org/wiki/Lov%C3%A1sz_conjecture#Hamiltonian_cycle) that every vertex transitive graph is hamiltonian. Then `conjecture(G)` should return `True` for every graph `G`. We can to test in on various small graphs drawn in the widget. If the graph in the above widget is still the Petersen graph, the following should return `False`, disproving the conjecture.

In [ ]:
conjecture(editor2.get_graph())

### Application 2: producing pictures for your papers

The drawing of the graph in the editor can be exported to a latex (tikz) picture to be included in a paper. The latex code can be obtained as follows: 

In [ ]:
latex(editor2.graph)

Note that only the positions of the vertices will be kept. See [this page](https://doc.sagemath.org/html/en/tutorial/latex.html#an-example-combinatorial-graphs-with-tkz-graph) for more details about exporting graphs to latex . The resulting pdf image can be seen as follows.

In [ ]:
view(editor2.graph)

The above requires ``pdflatex``. It  will fail if you run this demo on binder.

## Widget settings

Several parameters of the widget can be changed:
  * the width and height of the drawing canvas;
  * the default radius and color for vertices;
  * the default color for edges;
  * whether or not the display vertex and edge labels.

In [ ]:
editor3 = GraphEditor(graphs.PetersenGraph(), width=300, height=300, default_radius=12, default_vertex_color='orange', default_edge_color='#666', show_vertex_labels=False)
editor3.show()

## Changing the drawing

Changes to the drawing can be done with the mouse of course, but also by calling appropriate functions.

### Automatically setting positions

In [3]:
K = graphs.RandomBipartite(5,5,0.75)
editor4 = GraphEditor(K, width = 500, height = 500)
editor4.show()

The graph drawn above is bipartite and the partition numbe of a vertex is the first coordinate of its label. The code below sorts and colors vertices according to their partition number and changes their size according to the second coordinate of their label.

In [4]:
for v in editor4.graph:
    p, i = v
    
    editor4.set_vertex_radius(v, 3 * i + 25)
    if p:
        editor4.set_vertex_pos(v, 100, 50 +  100 * i)
        # editor4.set_vertex_color(v, 'red')
    else:
        editor4.set_vertex_pos(v, 400, 50 + 100 * i)
        #editor4.set_vertex_color(v, 'lightblue')
    
editor4.refresh()                    # needed to update the canvas

The names of the vertices are by default the vertex labels, but can be changed by redefining the `get_vertex_label` function.

In [9]:
def label(v):
    p, i = v
    return ('left ' if p else 'right ') + str(i)

editor4.get_vertex_label = label
editor4.refresh()

### Automatically setting colors

The colors of the vertices and edges can also be defined by a function.

In [10]:
n = 10
g = graphs.GridGraph([n,n])
editor5 = GraphEditor(g, default_radius=15, default_vertex_color='white')
editor5.show()

The code below recolors the vertices depending on their distance to vertex `(3,1)`.

In [12]:
def col(i, n):
    # Return a color depending on i
    rgbv = int(i * 255 / n)
    return '#%02x%02x%02x' % (100, rgbv , 255 - rgbv)

x,y = (3, 1) # source
# maximum distance to (x,y) in the graph
max_dist = max(x, n - 1 - x) + max(y, n - 1 -y)

for v in g:
    d = g.distance((x, y), v)
    editor5.set_vertex_color(v, col(d, max_dist))
editor5.refresh()

## Making animations

We now define a function that runs Dikjstra's algorithm on the graph of a `GraphEditor` widget and colors edges and vertices during when they are considered.

In [13]:
from random import randint, choice
from time import sleep

def wait():
    sleep(float(0.5))

def widget_dikjstra(w, source):
    # Adapted from the pseudocode at https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Pseudocode
    G = w.graph
    Q = G.vertices()
    prev = {v: None for v in G}
    dist = {v: 10000 for v in G}  # 10000 aka infinity
    dist[source] = 0
    
    while Q:
        u = Q[0]
        
        for v in Q:
            if dist[v] < dist[u]:
                u = v
        Q.remove(u)
        w.set_vertex_color(u, 'red')
        w.refresh()
        wait()
        
        for v in G.neighbor_iterator(u):
            if v not in Q:
                continue
            alt = dist[u] + G.edge_label(u,v)
            if alt < dist[v]:
                # update
                if prev[v] is not None:
                    w.set_edge_color((v, prev[v]), 'cyan')
                
                dist[v] = alt
                prev[v] = u
                w.set_vertex_color(v, 'green')
                w.set_edge_color((u,v), 'orange')
            else:
                w.set_edge_color((u,v), 'lightgray')
            w.refresh()
            wait()
        w.set_vertex_color(u, 'orange')
        if u == source:
            w.set_vertex_color(u, 'purple')
    w.refresh()

In [14]:
g = graphs.GridGraph([5,5])
# Give random labels to edges
for u,v in g.edge_iterator(labels=False):
    g.set_edge_label(u,v, randint(0,50))

editor6 = GraphEditor(g, default_radius=15, default_vertex_color='white', show_vertex_labels=False)
editor6.show()

The following runs Dikjstra's algorithm on the graph above, where edge labels represent distances
A vertex is:

  * white if it has not been discovered yet;
  * green if it has been discovered and not processed yet;
  * red if it is being processed;
  * orange if it has been processed;
  * purple if it is the source.

An edge is:

  * black if it has not been considered;
  * orange if it is currently part of a shortest path;
  * blue if it has been shortcut;
  * grey if it has been considered and is not a shortcut.

In [15]:
widget_dikjstra(editor6, choice(editor6.graph.vertices()))

## Running an algorithm step by step

A drawback of the animation described in the previous section is that one cannot pause it, which could be convenient in order to explain how the algorithm goes from one step to the next one. We show here how one can present the BFS algorithm step by step with the `GraphEditor` widget. For this we define below a function `widget_BFS` that returns a generator. Each time an element is extracted from this generator, a new step of the algorithm is ran and the drawing is updated. We can then define a button widget to run the algorithm step by step. This can also be used to test and debug an algorithm on specific graphs.

In [16]:
def widget_BFS(w, source):
    
    G = w.graph
    queue = [source]
    prev = {v: None for v in G}
    prev[source] = source
    
    while queue:
        
        # Take a new vertex in the queue
        v = queue.pop(0)
        w.set_vertex_color(v, 'red')
        w.refresh()
        yield None
        
        # Add all its neighbors to the queue if they have not already been considered
        for u in w.graph.neighbor_iterator(v):
            if prev[u] is not None: # u has already been seen
                if prev[v] != u and not w.get_edge_color((u,v)) == 'lightgray':  
                    w.set_edge_color((u,v), 'lightgray')
                    w.refresh()
                    yield None
            else:
                queue.append(u)
                prev[u] = v
                w.set_vertex_color(u, 'green')
                w.set_edge_color((u,v), 'orange')
                w.refresh()
                yield None

        if v is source:
            w.set_vertex_color(v, 'purple')
        else:
            w.set_vertex_color(v, 'orange')
    w.set_vertex_color(v, 'orange')
    w.refresh()

In [17]:
editor6 = GraphEditor(graphs.GridGraph([4,4]), default_radius=20, default_vertex_color='white', show_vertex_labels=False)

gen = widget_BFS(editor6, source=choice(editor6.graph.vertices()))

Now we define a button widget to run the algorithm

In [18]:
from ipywidgets import Button

button = Button(
    description='Next step',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='next step',
    icon='forward' # (FontAwesome names without the `fa-` prefix)
)

def button_clbk(b):
    try:
        next(gen)
    except StopIteration:
        b.disabled = True

# tie the button to button_clbk
button.on_click(button_clbk)

In [19]:
editor6.show()

In [20]:
button 

Button(description='Next step', icon='forward', style=ButtonStyle(), tooltip='next step')

In the drawing above, a vertex is:

  * white if it has not been discovered yet
  * green if it belongs to the queue (discovered and not processed yet)
  * red if it is being processed
  * orange if it has been processed
  * purple if it is the source.
  
An edge is

  * black if it has not been traversed yet
  * orange if it has been traversed to see a new vertex
  * gray if it has been traversed to see an already known vertex.

## Bonus: step-by-step Dijkstra with distances

In [25]:
def step_by_step_dijkstra(w, source):
    # Adapted from the pseudocode at https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Pseudocode
    G = w.graph
    Q = G.vertices()
    prev = {v: None for v in G}
    w.dist = {v: '∞' for v in G}
    w.dist[source] = 0
    
    while Q:
        u = Q[0]
        
        for v in Q:
            if w.dist[v] == '∞':
                continue
            if w.dist[u] == '∞' or w.dist[v] < w.dist[u]:
                u = v
        Q.remove(u)
        w.set_vertex_color(u, 'red')
        w.refresh()
        yield
        
        for v in G.neighbor_iterator(u):
            if v not in Q:
                continue
            alt = w.dist[u] + G.edge_label(u,v)
            if w.dist[v] == '∞' or alt < w.dist[v]:
                # update
                if prev[v] is not None:
                    w.set_edge_color((v, prev[v]), 'cyan')
                
                w.dist[v] = alt
                prev[v] = u
                w.set_vertex_color(v, 'green')
                w.set_edge_color((u,v), 'orange')
            else:
                w.set_edge_color((u,v), 'lightgray')
            w.refresh()
            yield
        w.set_vertex_color(u, 'orange')
        if u == source:
            w.set_vertex_color(u, 'purple')
    w.refresh()

In [32]:
g = graphs.GridGraph([5,5])
# Give random labels to edges
for u,v in g.edge_iterator(labels=False):
    g.set_edge_label(u,v, randint(0,50))

editor7 = GraphEditor(g, default_radius=20, default_vertex_color='white')
editor7.dist = {v: None for v in editor6.graph}

editor7.get_vertex_label = lambda v: str(editor7.dist[v])

button = Button(
    description='Next step',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='next step',
    icon='forward' # (FontAwesome names without the `fa-` prefix)
)

gen = step_by_step_dijkstra(editor7, choice(editor7.graph.vertices()))

def button_clbk(b):
    try:
        next(gen)
    except StopIteration:
        b.disabled = True

# tie the button to button_clbk
button.on_click(button_clbk)

In [33]:
editor7.show()

In [34]:
button

Button(description='Next step', icon='forward', style=ButtonStyle(), tooltip='next step')